In [14]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup as bs
from selenium.webdriver.support import expected_conditions
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
import time
import pandas as pd

In [3]:
chrome_driver_path = "Y:\\chromedriver-win64\\chromedriver.exe"

chrome_options = Options()
chrome_options.binary_location = "Y:\\chrome-win64\\chrome.exe"
service = Service(chrome_driver_path)

In [4]:
driver = webdriver.Chrome(service=service,options=chrome_options)
driver.maximize_window()
driver.get('https://amrtamya.com/')

In [ ]:
# To About Page
WebDriverWait(driver,6).until(expected_conditions.element_to_be_clickable((By.ID,"HeaderMenu-about-us"))).click()
About_page = driver.find_element(By.ID,'shopify-section-template--19393857716447__main')
print(About_page.text)

# To shop page
WebDriverWait(driver,2).until(expected_conditions.element_to_be_clickable((By.ID,'HeaderMenu-shop'))).click()
all_item_shop = driver.find_elements(By.XPATH,'//*[@id="shopify-section-template--19393857650911__main"]/div/ul')
print(all_item_shop[0].text)

time.sleep(2)
list_of_oil_category = driver.find_elements(By.XPATH,'//*[@id="shopify-section-template--19393857650911__main"]/div/ul')[0]

scraped_data = {'Category':{},'Product_detail':{}}

for item in list_of_oil_category.find_elements(By.TAG_NAME,'li'):
    # Get the link element
    link_elem = item.find_element(By.TAG_NAME, 'a')
    link = link_elem.get_attribute('href')
    name = item.text
    
    # Get the image element (if present)
    try:
        img_elem = item.find_element(By.TAG_NAME, 'img')
        img_src = img_elem.get_attribute('src')
    except:
        img_src = None
    
    scraped_data['Category'].update({name: {'link':link , 'image':img_src}})

    print(f"Name: {name}")
    print(f"Link: {link}")
    print(f"Image: {img_src}")
    print('-' * 40)

time.sleep(2)
main_window = driver.current_window_handle

for key,value in scraped_data['Category'].items():
    category_link = value['link']

    driver.execute_script(f"window.open('{category_link}', '_blank');")
    time.sleep(3)
    
    driver.switch_to.window(driver.window_handles[-1])
    
    # About Category
    try:
        About_category = driver.find_element(By.CLASS_NAME,'collection-hero__description rte').text
    except:
        About_category = None
    scraped_data['Category'][key].update({'About_category': About_category})

    # At page, product list
    WebDriverWait(driver, 10).until(expected_conditions.presence_of_element_located((By.CLASS_NAME, 'product-grid')))

    # Products under Each Category
    products = driver.find_elements(By.CSS_SELECTOR, ".product-grid .grid__item")

    for product in products:
        link_ele = product.find_element(By.TAG_NAME, "a")
        product_link = link_ele.get_attribute("href")
        # Go to product link
        driver.execute_script(f"window.open('{product_link}', '_blank');")
        time.sleep(5)
        driver.switch_to.window(driver.window_handles[-1])
        time.sleep(3)
        
        # Product Name
        name = driver.find_element(By.CLASS_NAME, "product__title").text

        # Description
        try:
            product_desc = driver.find_element(By.CSS_SELECTOR, ".product__description.rte.quick-add-hidden").text
        except:
            product_desc = None

        # Review/Ratings
        try:
            reviews = driver.find_element(By.CLASS_NAME,'jdgm-prev-badge__text').text
        except:
            reviews = None
        
        try:
            ratings = driver.find_element(By.CLASS_NAME,'jdgm-rev-widg__summary-average').text
        except:
            ratings = None

        # Price
        try:
            price_ele = driver.find_element(By.CLASS_NAME, "price__container")
            price_regular = price_ele.find_element(By.CLASS_NAME, "price__regular").text
        except:
            price_regular = None
        
        # Size
        try:
            size_options = []
            size_ele = driver.find_element(By.ID,'variant-selects-template--19393857781983__main')
            sizes = size_ele.find_elements(By.TAG_NAME ,'input')
            for quantity in sizes:
                size_options.append(quantity.get_attribute('value'))
        except:
            size_options = None    
        
        # Image
        try:
            product_images = []
            image_ele_grid = driver.find_element(By.CLASS_NAME,'product__column-sticky')
            images = image_ele_grid.find_elements(By.TAG_NAME,'img')
            for img in images:
                product_images.append(img.get_attribute('src'))
        except:
            product_images = None

        try:    
            customer_reviews = driver.find_element(By.CLASS_NAME,'jdgm-rev-widg__body').text
        except:
            customer_reviews = None

        time.sleep(1)
        # Recomendations
        product_recomendation = []
        recommendations = driver.find_elements(By.CSS_SELECTOR, "ul.grid.product-grid > li")
        for rec in recommendations:
            # Product name and link
            try:
                a_tag = rec.find_element(By.CSS_SELECTOR, "a.full-unstyled-link")
                rec_name = a_tag.text.strip()
                rec_link = a_tag.get_attribute("href")
            except:
                rec_name = None
                rec_link = None
            # Product image
            try:
                rec_img = rec.find_element(By.CSS_SELECTOR, "img").get_attribute("src")
            except:
                rec_img = None
            # Product price
            try:
                rec_price = rec.find_element(By.CSS_SELECTOR, ".price-item--regular").text
            except:
                rec_price = None
            product_recomendation.append({'Product_Name':rec_name,
             'Rec_Link':rec_link,
             'Product_Image':rec_img,
             'Price':rec_price})
    
        scraped_data['Product_detail'].update({name:{'Price':price_regular, 
                                                    'Under_Category':key, 
                                                    'Product_link':product_link, 
                                                    'Description':product_desc, 
                                                    'Number_of_Reviews':reviews, 
                                                    'Total_Ratings':ratings, 
                                                    'Avaialble_in':size_options,
                                                    'Product_Images':product_images,
                                                    'Customer_Review':customer_reviews,
                                                    'Recommendations':product_recomendation
                                                    }})
        time.sleep(1)
        driver.close()  # Close product tab
        driver.switch_to.window(driver.window_handles[-1])  # Switch back to category tab

    driver.close()  # Close category tab
    driver.switch_to.window(main_window)  # Switch back to main window
driver.switch_to.window(driver.window_handles[0])

# Contact Information
WebDriverWait(driver,6).until(expected_conditions.element_to_be_clickable((By.ID,'HeaderMenu-contact-us'))).click()
Contact_Info = driver.find_element(By.ID,'shopify-section-template--19393857716447__main').text
scraped_data['Contact_Information'] = Contact_Info

About us
Amrtamya, the journey began with love for nature and the natural.
The dream of building a business that generates a by product that’s all natural and useful for growing natural.
The calm and peace of the slow process is far from our professional backgrounds, but the one that’s bringing us closer to people, real problems and making us more sensitive.
Our story is stories of our customers who bring back with them the experience and experiments of power of nature. The healing of body and mind is "all within" and "in nature".
Our journey has begun with the cold pressed oil. A very important ingredient of the day, a product that’s available in the market but bought with partial satisfaction. We bring this satisfaction with experience of live extraction.
The challenges of manufacturing a product that’s chemical and preservatives free are many, as almost a whole generation has grown up on uniform, standard quality. But the nature is different.
Our product has "NO" added chemicals, "N

TypeError: element_to_be_clickable() takes 1 positional argument but 2 were given

In [12]:
scraped_data

{'Category': {'Wellness Oils': {'link': 'https://amrtamya.com/collections/wellness-oils',
   'image': 'https://amrtamya.com/cdn/shop/collections/FullSizeRender.heic?v=1747669810&width=1500',
   'About_category': None},
  'Edible Oils': {'link': 'https://amrtamya.com/collections/edible-oils',
   'image': 'https://amrtamya.com/cdn/shop/collections/White-Sesame_1_Litre-Photoroom.jpg?v=1747675066&width=1500',
   'About_category': None},
  'Natural Distilled Water': {'link': 'https://amrtamya.com/collections/natural-distilled-water',
   'image': 'https://amrtamya.com/cdn/shop/collections/All_Distilled_Water.jpg?v=1708786967&width=1500',
   'About_category': None},
  'Essential oils and Attars': {'link': 'https://amrtamya.com/collections/essential-oils-and-attars',
   'image': 'https://amrtamya.com/cdn/shop/collections/IMG-4515.jpg?v=1747675953&width=1500',
   'About_category': None},
  'After-Bath Body oils': {'link': 'https://amrtamya.com/collections/after-bath-body-oils',
   'image': 'htt

In [ ]:
rows = []
for product_name, details in scraped_data['Product_detail'].items():
    row = {
        'Product Name': product_name,
        'Category': details.get('Under_Category'),
        'Product Link': details.get('Product_link'),
        'Price': details.get('Price'),
        'Description': details.get('Description'),
        'Number of Reviews': details.get('Number_of_Reviews'),
        'Total Ratings': details.get('Total_Ratings'),
        'Available In': ', '.join(details.get('Avaialble_in', []) or []),
        'Product Images': ', '.join(details.get('Product_Images', []) or []),
        'Customer Review': details.get('Customer_Review'),
    }
    # Flatten recommendations if present
    recs = details.get('Recommendations', [])
    if isinstance(recs, list) and recs:
        # Take only the first recommendation for simplicity, or join all
        rec_names = [r.get('Product_Name', '') for r in recs]
        rec_links = [r.get('Rec_Link', '') for r in recs]
        rec_imgs = [r.get('Product_Image', '') for r in recs]
        rec_prices = [r.get('Price', '') for r in recs]
        row['Rec Product Names'] = ', '.join(rec_names)
        row['Rec Product Links'] = ', '.join(rec_links)
        row['Rec Product Images'] = ', '.join(rec_imgs)
        row['Rec Product Prices'] = ', '.join(rec_prices)
    rows.append(row)

# Create DataFrame and save to CSV
df = pd.DataFrame(rows)
df.to_csv('amrtamya_products.csv', index=False)